# Query Translation: Step Back

![step-back](../images/images-step-back.png)

**Step-Back** is a technique that works by broadening a query to retrieve more general, high-level information before refining it for a specific answer. It takes a "step back" to provide the system with broader context, ensuring more complete and accurate results.

### The Problem Step-Back Solves
Sometimes, user queries can be overly specific or lack sufficient context, which may:
- Limit the retrieval to only narrow or irrelevant information.
- Prevent the system from understanding the query’s broader intent.

**Step-Back** solves this by generating a more general version of the query first, retrieving broader context, and then using that context to refine the retrieval or generation process.

### How Step-Back Works
1. **Analyze the Query**:
   - Identify if the query is too specific, lacks context, or could benefit from a broader starting point.
   - Example:
     - Original Query: "How does OAuth 2.0 work in securing APIs?"

2. **Generate a Generalized Query**:
   - Rewrite the original query to be more general or abstract, focusing on the broader topic.
   - Example:
     - General Query: "What are the general principles of securing APIs?"

3. **Retrieve Broader Context**:
   - Use the generalized query to retrieve high-level or foundational information about the topic.
   - Example:
     - Documents retrieved might include general API security practices, authentication methods, and common threats.

4. **Refine the Response**:
   - Use the retrieved broader context to either:
     - Directly enhance the response generation.
     - Refine the retrieval step further by adding the original specific query for a targeted follow-up.
   - Example:
     - Combine "general principles of securing APIs" with specifics about OAuth 2.0.

5. **Generate the Final Output**:
   - The combined context (broader and specific) is passed to the LLM to generate a well-informed, grounded response.

### Why Step-Back Works
- **Broadens Context**: Ensures the system has a comprehensive understanding of the topic before diving into specifics.
- **Improves Retrieval**: Prevents overly specific queries from missing important related information.
- **Grounds Generation**: Helps the system produce well-rounded and accurate responses.

### Simple Example
#### Query:
*"How does token expiry work in OAuth 2.0 for API security?"*

#### Without Step-Back:
- The system might retrieve overly specific documents on token expiry without addressing the broader context of why token expiry is important in API security.

#### With Step-Back:
1. Generalized Query:
   - "What are the security features of OAuth 2.0?"
2. Retrieved Documents:
   - General documents on OAuth 2.0, covering concepts like token-based authentication, refresh tokens, and expiry policies.
3. Refine with Original Query:
   - Use the specific query "How does token expiry work?" to find more detailed information.
4. Combined Context:
   - Broader understanding of OAuth 2.0 security combined with specific details about token expiry.
5. Final Output:
   - "OAuth 2.0 provides token-based authentication to secure APIs. Token expiry ensures that access is time-limited, reducing the risk of token misuse. Refresh tokens can be used to issue new access tokens without compromising security."

### Key Benefits of Step-Back
- **Ensures Completeness**: Provides foundational knowledge before diving into specifics.
- **Enhances Retrieval**: Prevents the system from missing key information by starting broad.
- **Reduces Errors**: Avoids misinterpretations that can occur when overly specific queries lack context.

In short, **Step-Back** is like zooming out to see the bigger picture before focusing on the finer details, ensuring a more complete and accurate response!

![step back](../images/step%20back.png)

Arxiv papers:

- [Take a Step Back: Evoking Reasoning via Abstraction in Large Language Models](https://arxiv.org/pdf/2310.06117)

## Setup

In [1]:
%run "../Z - Common/setup.ipynb"

Stored 'enable_langsmith' (bool)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
docs = load_sample_data()
split_docs = split_sample_data(docs)
retriever = seed_sample_data(split_docs)

Let's start by defining a prompt which includes few shot examples, and the chain to generate the step backed queries:

In [4]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [5]:
from langchain_core.output_parsers import StrOutputParser

chain_generate_queries = prompt | llm | StrOutputParser()


In [6]:
question = "What are the main components of an LLM-powered autonomous agent system?"
chain_generate_queries.invoke({"question": question})

'what are the key parts of an AI system?'

Define and execute the overall chain:

In [7]:
from langchain_core.runnables import RunnableLambda

# Response prompt 
template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": chain_generate_queries | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)



In [9]:
result=chain.invoke({"question": question})
result

'Based on the provided context, an LLM-powered autonomous agent system has the following main components:\n\n1. LLM as the Core Controller (Brain)\n- Functions as the central processing unit or "brain" of the agent system\n- Goes beyond basic text generation to serve as a general problem solver\n\n2. Planning Component\nThis includes two key aspects:\n- Subgoal and decomposition\n  * Breaks down large tasks into smaller, manageable subgoals\n  * Enables efficient handling of complex tasks\n- Reflection and refinement\n  * Allows for self-criticism and self-reflection on past actions\n  * Learns from mistakes\n  * Refines actions for future steps\n  * Improves the quality of final results\n\n3. Memory Component\nWhile the specific details of the memory component aren\'t elaborated in the given context, it is mentioned as one of the key components of the system.\n\nThe context also mentions examples of such systems like AutoGPT, GPT-Engineer, and BabyAGI as proof-of-concept demonstration

Save the results so we can compare later.

In [10]:
write_results("step-back.txt", result)